<center><p float="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/4_RGB_McCombs_School_Brand_Branded.png" width="300" height="100"/>
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<h1><center><font size=10>Artificial Intelligence and Machine Learning</center></font></h1>
<h1><center>Introduction to Natural Language Processing - Project Debrief</center></h1>

<center><p float="center">
  <img src="https://cdn.pixabay.com/photo/2017/07/24/04/23/technical-support-2533526_1280.png" width="480"/>
</p></center>

<center><font size=5>Support Ticket Categorization</center></font>

## Note: This is a sample solution for the project. Projects will NOT be graded on the basis of how well the submission matches this sample solution. Projects will be graded on the basis of the rubric only.

## Problem Statement

### Business Context

In today's dynamic business landscape, organizations are increasingly recognizing the pivotal role customer feedback plays in shaping the trajectory of their products and services. The ability to swiftly and effectively respond to customer input not only fosters enhanced customer experiences but also serves as a catalyst for growth, prolonged customer engagement, and the nurturing of lifetime value relationships. As a dedicated Product Manager or Product Analyst, staying attuned to the voice of your customers is not just a best practice; it's a strategic imperative.

While your organization may be inundated with a wealth of customer-generated feedback and support tickets, your role entails much more than just processing these inputs. To make your efforts in managing customer experience and expectations truly impactful, you need a structured approach – a method that allows you to discern the most pressing issues, set priorities, and allocate resources judiciously. One of the most effective strategies at your disposal is to harness the power of Support Ticket Categorization.


### Objective

Develop an advanced support ticket categorization system that accurately classifies incoming tickets, assigns relevant tags based on their content, implements mechanisms and generate the first response based on the sentiment for prioritizing tickets for prompt resolution.


## Installing and Importing Necessary Libraries and Dependencies

In [ ]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 141.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu

**Note** : There may be an error related to a dependency issue thrown by the pip package. This can be ignored as it will not impact the execution of the code.

In [ ]:
# For downloading the models from HF Hub
!pip install huggingface_hub==0.20.3 pandas==1.5.3 -q

In [ ]:
# Function to download the model from the Hugging Face model hub
from huggingface_hub import hf_hub_download

# Importing the Llama class from the llama_cpp module
from llama_cpp import Llama

# Importing the json module
import json

# for loading and manipulating data
import pandas as pd

# for time computations
import time

## Loading the Data

In [ ]:
data = pd.read_csv("/content/support_ticket_data.csv")

## Data Overview

### Checking the first 5 rows of the data

In [ ]:
data.head()

,support_tick_id,support_ticket_text
0,ST2023-006,My internet connection has significantly slowe...
1,ST2023-007,Urgent help required! My laptop refuses to sta...
2,ST2023-008,I've accidentally deleted essential work docum...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...
4,ST2023-010,"My smartphone battery is draining rapidly, eve..."


### Checking the shape of the data

In [ ]:
data.shape

(21, 2)

* There are 21 rows and 2 columns

### Checking the missing values in the data

In [ ]:
data.isnull().sum()

support_tick_id        0
support_ticket_text    0
dtype: int64

* There are no missing values in the data

## Model Building

### Loading the model

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

In [ ]:
#uncomment the below snippet of code if the runtime is connected to GPU.
llm = Llama(
    model_path=model_path,
    n_ctx=1024,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
#uncomment the below snippet of code if the runtime is connected to CPU only.
#llm = Llama(
#    model_path=model_path,
#    n_ctx=1024,
#    n_cores=-2
#)

### Utility function

In [ ]:
# defining a function to parse the JSON output from the model
def extract_json_data(json_str):
    try:
        # Find the indices of the opening and closing curly braces
        json_start = json_str.find('{')
        json_end = json_str.rfind('}')

        if json_start != -1 and json_end != -1:
            extracted_category = json_str[json_start:json_end + 1]  # Extract the JSON object
            data_dict = json.loads(extracted_category)
            return data_dict
        else:
            print(f"Warning: JSON object not found in response: {json_str}")
            return {}
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {}

## Task 1: Ticket Categorization and Returning Structured Output

In [ ]:
#Defining the response funciton for Task 1.
def response_1(prompt,ticket):
    start = time.time()
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      A:
      """,
      max_tokens=32,
      stop=["Q:", "\n"],
      temperature=0.01,
      echo=False,
    )
    end = time.time()

    temp_output = model_output["choices"][0]["text"]
    final_output = temp_output[temp_output.index('{'):]

    return final_output

In [ ]:
# creating a copy of the data
data_1 = data.copy()

In [ ]:
prompt_1 = """
    You are a technical assistant. Classify the support ticket based on the Support Ticket Text presented
    in the input into the following categories and not any other.
      - Technical issues
      - Hardware issues
      - Data recovery
    Return only a structured JSON output in the following format:
    {"Category": "category_prediction"}
"""

In [ ]:
start = time.time()
data_1['model_response'] = data_1['support_ticket_text'].apply(lambda x: response_1(prompt_1, x))
end = time.time()

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


In [ ]:
print("Time taken ",(end-start))

Time taken  319.34698700904846


In [ ]:
data_1['model_response']

0     {"Category": "Technical issues"}
1      {"Category": "Hardware issues"}
2        {"Category": "Data recovery"}
3     {"Category": "Technical issues"}
4     {"Category": "Technical issues"}
5     {"Category": "Technical issues"}
6     {"Category": "Technical issues"}
7      {"Category": "Hardware issues"}
8        {"Category": "Data recovery"}
9      {"Category": "Hardware issues"}
10       {"Category": "Data recovery"}
11     {"Category": "Hardware issues"}
12       {"Category": "Data recovery"}
13       {"Category": "Data recovery"}
14     {"Category": "Hardware issues"}
15    {"Category": "Technical issues"}
16    {"Category": "Technical issues"}
17       {"Category": "Data recovery"}
18       {"Category": "Data recovery"}
19    {"Category": "Technical issues"}
20    {"Category": "Technical issues"}
Name: model_response, dtype: object

In [ ]:
i = 2
print(data_1.loc[i, 'support_ticket_text'])

I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?


In [ ]:
print(data_1.loc[i, 'model_response'])

{"Category": "Data recovery"}


In [ ]:
# applying the function to the model response
data_1['model_response_parsed'] = data_1['model_response'].apply(extract_json_data)
data_1['model_response_parsed'].head()

0    {'Category': 'Technical issues'}
1     {'Category': 'Hardware issues'}
2       {'Category': 'Data recovery'}
3    {'Category': 'Technical issues'}
4    {'Category': 'Technical issues'}
Name: model_response_parsed, dtype: object

In [ ]:
data_1['model_response_parsed'].value_counts()

{'Category': 'Technical issues'}    9
{'Category': 'Data recovery'}       7
{'Category': 'Hardware issues'}     5
Name: model_response_parsed, dtype: int64

In [ ]:
# Normalizing the model_response_parsed column
model_response_parsed_df_1 = pd.json_normalize(data_1['model_response_parsed'])
model_response_parsed_df_1.head()

,Category
0,Technical issues
1,Hardware issues
2,Data recovery
3,Technical issues
4,Technical issues


In [ ]:
# Concatinating two dataframes
data_with_parsed_model_output_1 = pd.concat([data_1, model_response_parsed_df_1], axis=1)
data_with_parsed_model_output_1.head()

,support_tick_id,support_ticket_text,model_response,model_response_parsed,Category
0,ST2023-006,My internet connection has significantly slowe...,"{""Category"": ""Technical issues""}",{'Category': 'Technical issues'},Technical issues
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{""Category"": ""Hardware issues""}",{'Category': 'Hardware issues'},Hardware issues
2,ST2023-008,I've accidentally deleted essential work docum...,"{""Category"": ""Data recovery""}",{'Category': 'Data recovery'},Data recovery
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{""Category"": ""Technical issues""}",{'Category': 'Technical issues'},Technical issues
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","{""Category"": ""Technical issues""}",{'Category': 'Technical issues'},Technical issues


In [ ]:
# Dropping model_response and model_response_parsed columns
final_data_1 = data_with_parsed_model_output_1.drop(['model_response','model_response_parsed'], axis=1)
final_data_1.head()

,support_tick_id,support_ticket_text,Category
0,ST2023-006,My internet connection has significantly slowe...,Technical issues
1,ST2023-007,Urgent help required! My laptop refuses to sta...,Hardware issues
2,ST2023-008,I've accidentally deleted essential work docum...,Data recovery
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Technical issues
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Technical issues


## Task 2: Creating Tags

In [ ]:
# creating a copy of the data
data_2 = data.copy()

In [ ]:
def response_2(prompt,ticket,category):
    start = time.time()
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      Category: {category}
      A:
      """,
      max_tokens=64,
      stop=["Q:", "\n"],
      temperature=0.01,
      echo=False,
    )
    end = time.time()

    temp_output = model_output["choices"][0]["text"]

    final_output = temp_output[temp_output.index('{'):]

    return final_output

In [ ]:
prompt_2 = """
    Support ticket and Category are provided as inputs. Based on the Support ticket and Category provided for the ticket,
    identify the following:
      - Tags for the ticket.

    Instructions:
     Analyse the given Support ticket and identify the important keywords which describes the issue in the ticket.

    Return in this format {"Tags": "tags_prediction"}
"""

In [ ]:
start = time.time()
data_2["model_response"]=final_data_1[['support_ticket_text','Category']].apply(lambda x: response_2(prompt_2, x[0],x[1]),axis =1)
end = time.time()

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


In [ ]:
print("Time taken ",end-start)

Time taken  472.5123965740204


In [ ]:
data_2['model_response'].values

array(['{"Tags": ["Internet", "Connection", "Slowdown", "Disconnection"]}',
       '{"Tags": ["Urgent", "Hardware failure", "Laptop start-up", "Presentation"]}',
       '{"Tags": "data_loss, document, recovery"}',
       '{"Tags": ["Wi-Fi", "Signal strength", "Troubleshooting"]}',
       '{"Tags": ["Battery", "Smartphone", "Draining", "Technical Issues"]}',
       '{"Tags": ["Account Access", "Password Reset", "Online Banking", "Technical Issue"]}',
       '{"Tags": ["performance", "optimization", "computer", "work"]}',
       '{"Tags": ["Blue Screen", "Crash", "Hardware Issue"]}',
       '{"Tags": ["Data Recovery", "External Hard Drive"]}',
       '{"Tags": ["Graphics Card", "Hardware Issue", "Gaming Performance"]}',
       '{"Tags": ["Data loss", "Data recovery", "USB"]}',
       '{"Tags": ["computer", "screen", "blackout", "hardware", "urgent"]}',
       '{"Tags": ["Water Damage", "Data Recovery"]}',
       '{"Tags": ["Data Recovery", "USB Flash Drive", "Physical Damage"]}',
       

In [ ]:
i = 2
print(data_2.loc[i, 'support_ticket_text'])

I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?


In [ ]:
print(data_2.loc[i, 'model_response'])

{"Tags": "data_loss, document, recovery"}


In [ ]:
# Applying the function to the model response
data_2['model_response_parsed'] = data_2['model_response'].apply(extract_json_data)

In [ ]:
data_2["model_response_parsed"]

0     {'Tags': ['Internet', 'Connection', 'Slowdown'...
1     {'Tags': ['Urgent', 'Hardware failure', 'Lapto...
2             {'Tags': 'data_loss, document, recovery'}
3     {'Tags': ['Wi-Fi', 'Signal strength', 'Trouble...
4     {'Tags': ['Battery', 'Smartphone', 'Draining',...
5     {'Tags': ['Account Access', 'Password Reset', ...
6     {'Tags': ['performance', 'optimization', 'comp...
7     {'Tags': ['Blue Screen', 'Crash', 'Hardware Is...
8     {'Tags': ['Data Recovery', 'External Hard Driv...
9     {'Tags': ['Graphics Card', 'Hardware Issue', '...
10      {'Tags': ['Data loss', 'Data recovery', 'USB']}
11    {'Tags': ['computer', 'screen', 'blackout', 'h...
12          {'Tags': ['Water Damage', 'Data Recovery']}
13    {'Tags': ['Data Recovery', 'USB Flash Drive', ...
14         {'Tags': ['touchpad', 'laptop', 'hardware']}
15    {'Tags': ['Internet', 'Connection', 'Drop', 'F...
16    {'Tags': ['Wi-Fi', 'Connectivity', 'Router', '...
17      {'Tags': ['Data loss', 'Data recovery', 

In [ ]:
# Normalizing the model_response_parsed column
model_response_parsed_df_2 = pd.json_normalize(data_2['model_response_parsed'])
model_response_parsed_df_2.head()

,Tags
0,"[Internet, Connection, Slowdown, Disconnection]"
1,"[Urgent, Hardware failure, Laptop start-up, Pr..."
2,"data_loss, document, recovery"
3,"[Wi-Fi, Signal strength, Troubleshooting]"
4,"[Battery, Smartphone, Draining, Technical Issues]"


In [ ]:
# Concatinating two dataframes
data_with_parsed_model_output_2 = pd.concat([data_2, model_response_parsed_df_2], axis=1)
data_with_parsed_model_output_2.head()

,support_tick_id,support_ticket_text,model_response,model_response_parsed,Tags
0,ST2023-006,My internet connection has significantly slowe...,"{""Tags"": [""Internet"", ""Connection"", ""Slowdown""...","{'Tags': ['Internet', 'Connection', 'Slowdown'...","[Internet, Connection, Slowdown, Disconnection]"
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{""Tags"": [""Urgent"", ""Hardware failure"", ""Lapto...","{'Tags': ['Urgent', 'Hardware failure', 'Lapto...","[Urgent, Hardware failure, Laptop start-up, Pr..."
2,ST2023-008,I've accidentally deleted essential work docum...,"{""Tags"": ""data_loss, document, recovery""}","{'Tags': 'data_loss, document, recovery'}","data_loss, document, recovery"
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{""Tags"": [""Wi-Fi"", ""Signal strength"", ""Trouble...","{'Tags': ['Wi-Fi', 'Signal strength', 'Trouble...","[Wi-Fi, Signal strength, Troubleshooting]"
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","{""Tags"": [""Battery"", ""Smartphone"", ""Draining"",...","{'Tags': ['Battery', 'Smartphone', 'Draining',...","[Battery, Smartphone, Draining, Technical Issues]"


In [ ]:
# Dropping model_response and model_response_parsed columns
final_data_2 = data_with_parsed_model_output_2.drop(['model_response','model_response_parsed'], axis=1)
final_data_2.head()

,support_tick_id,support_ticket_text,Tags
0,ST2023-006,My internet connection has significantly slowe...,"[Internet, Connection, Slowdown, Disconnection]"
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"[Urgent, Hardware failure, Laptop start-up, Pr..."
2,ST2023-008,I've accidentally deleted essential work docum...,"data_loss, document, recovery"
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"[Wi-Fi, Signal strength, Troubleshooting]"
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","[Battery, Smartphone, Draining, Technical Issues]"


In [ ]:
# Checking the value counts of Category column
final_data_2['Tags'].value_counts()

[Data loss, Data recovery, USB]                                                     2
[Data Recovery, External Hard Drive]                                                2
[Internet, Connection, Slowdown, Disconnection]                                     1
data_loss, document, recovery                                                       1
[Urgent, Hardware failure, Laptop start-up, Presentation]                           1
[Wi-Fi, Signal strength, Troubleshooting]                                           1
[Battery, Smartphone, Draining, Technical Issues]                                   1
[performance, optimization, computer, work]                                         1
[Account Access, Password Reset, Online Banking, Technical Issue]                   1
[Blue Screen, Crash, Hardware Issue]                                                1
[Graphics Card, Hardware Issue, Gaming Performance]                                 1
[computer, screen, blackout, hardware, urgent]        

In [ ]:
final_data_2 = pd.concat([final_data_2,final_data_1["Category"]],axis=1)

In [ ]:
final_data_2 = final_data_2[["support_tick_id","support_ticket_text","Category","Tags"]]
final_data_2

,support_tick_id,support_ticket_text,Category,Tags
0,ST2023-006,My internet connection has significantly slowe...,Technical issues,"[Internet, Connection, Slowdown, Disconnection]"
1,ST2023-007,Urgent help required! My laptop refuses to sta...,Hardware issues,"[Urgent, Hardware failure, Laptop start-up, Pr..."
2,ST2023-008,I've accidentally deleted essential work docum...,Data recovery,"data_loss, document, recovery"
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Technical issues,"[Wi-Fi, Signal strength, Troubleshooting]"
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Technical issues,"[Battery, Smartphone, Draining, Technical Issues]"
5,ST2023-011,I'm locked out of my online banking account an...,Technical issues,"[Account Access, Password Reset, Online Bankin..."
6,ST2023-012,"My computer's performance is sluggish, severel...",Technical issues,"[performance, optimization, computer, work]"
7,ST2023-013,I'm experiencing a recurring blue screen error...,Hardware issues,"[Blue Screen, Crash, Hardware Issue]"
8,ST2023-014,My external hard drive isn't being recognized ...,Data recovery,"[Data Recovery, External Hard Drive]"
9,ST2023-015,The graphics card in my gaming laptop seems to...,Hardware issues,"[Graphics Card, Hardware Issue, Gaming Perform..."


## Task 3: Predicting Priority and ETA

In [ ]:
# creating a copy of the data
data_3 = data.copy()

In [ ]:
def response_3(prompt,ticket,category,tags):
    start = time.time()
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      Category: {category}
      Tags: {tags}
      A:
      """,
      max_tokens=64,
      stop=["Q:", "\n"],
      temperature=0.01,
      echo=False,
    )
    end = time.time()

    temp_output = model_output["choices"][0]["text"]
    final_output = temp_output[temp_output.index('{'):]

    return final_output

In [ ]:
prompt_3 = """
    You are provided a Support ticket,Category and Tags as input. Based on the text in the ticket,category and tags provided for the ticket,
    identify the following:
      - Priority of resolving the ticket
      - ETA for resolving the ticket

    Instructions:
    1. Carefully read the text of the support ticket and consider the overall sentiment of the review while assigning the priority
    2. The priority should be one of Low, Medium, or High only and not any other.
    3. All tickets cannot be of high priority.
    4. Depending on the sentiment of the review, the priority of the ticket, and the nature of the issue being faced by the customer, suggest an ETA for resolution.
    5. ETA can be one of Within 24 hours,Within 2-4 hours,Within 48 hours,ASAP only and not any other.

     Return in this format: {"Priority": "priority_prediction", "ETA": "eta_prediction"}
"""

In [ ]:
# Applying generate_llama_response function on support_ticket_text column
start = time.time()
data_3['model_response'] = final_data_2[['support_ticket_text','Category','Tags']].apply(lambda x: response_3(prompt_3, x[0],x[1],x[2]),axis=1)
end = time.time()

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


In [ ]:
print("Time taken ",(end-start))

Time taken  343.777804851532


In [ ]:
data_3['model_response']

0                  {"Priority": "High", "ETA": "ASAP"}
1                  {"Priority": "High", "ETA": "ASAP"}
2                  {"Priority": "High", "ETA": "ASAP"}
3     {"Priority": "Medium", "ETA": "Within 24 hours"}
4     {"Priority": "Medium", "ETA": "Within 24 hours"}
5                  {"Priority": "High", "ETA": "ASAP"}
6       {"Priority": "High", "ETA": "Within 24 hours"}
7                  {"Priority": "High", "ETA": "ASAP"}
8                  {"Priority": "High", "ETA": "ASAP"}
9       {"Priority": "High", "ETA": "Within 24 hours"}
10                 {"Priority": "High", "ETA": "ASAP"}
11                 {"Priority": "High", "ETA": "ASAP"}
12                 {"Priority": "High", "ETA": "ASAP"}
13                 {"Priority": "High", "ETA": "ASAP"}
14      {"Priority": "High", "ETA": "Within 48 hours"}
15                 {"Priority": "High", "ETA": "ASAP"}
16       {"Priority": "High", "ETA": "Within 4 hours"}
17                 {"Priority": "High", "ETA": "ASAP"}
18        

In [ ]:
i = 2
print(data_3.loc[i, 'support_ticket_text'])

I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?


In [ ]:
print(data_3.loc[i, 'model_response'])

{"Priority": "High", "ETA": "ASAP"}


In [ ]:
# Applying the function to the model response
data_3['model_response_parsed'] = data_3['model_response'].apply(extract_json_data)
data_3['model_response_parsed'].head()

0                 {'Priority': 'High', 'ETA': 'ASAP'}
1                 {'Priority': 'High', 'ETA': 'ASAP'}
2                 {'Priority': 'High', 'ETA': 'ASAP'}
3    {'Priority': 'Medium', 'ETA': 'Within 24 hours'}
4    {'Priority': 'Medium', 'ETA': 'Within 24 hours'}
Name: model_response_parsed, dtype: object

In [ ]:
# Normalizing the model_response_parsed column
model_response_parsed_df_3 = pd.json_normalize(data_3['model_response_parsed'])
model_response_parsed_df_3.head(21)

,Priority,ETA
0,High,ASAP
1,High,ASAP
2,High,ASAP
3,Medium,Within 24 hours
4,Medium,Within 24 hours
5,High,ASAP
6,High,Within 24 hours
7,High,ASAP
8,High,ASAP
9,High,Within 24 hours


In [ ]:
# Concatinating two dataframes
data_with_parsed_model_output_3 = pd.concat([data_3, model_response_parsed_df_3], axis=1)
data_with_parsed_model_output_3.head()

,support_tick_id,support_ticket_text,model_response,model_response_parsed,Priority,ETA
0,ST2023-006,My internet connection has significantly slowe...,"{""Priority"": ""High"", ""ETA"": ""ASAP""}","{'Priority': 'High', 'ETA': 'ASAP'}",High,ASAP
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{""Priority"": ""High"", ""ETA"": ""ASAP""}","{'Priority': 'High', 'ETA': 'ASAP'}",High,ASAP
2,ST2023-008,I've accidentally deleted essential work docum...,"{""Priority"": ""High"", ""ETA"": ""ASAP""}","{'Priority': 'High', 'ETA': 'ASAP'}",High,ASAP
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{""Priority"": ""Medium"", ""ETA"": ""Within 24 hours""}","{'Priority': 'Medium', 'ETA': 'Within 24 hours'}",Medium,Within 24 hours
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","{""Priority"": ""Medium"", ""ETA"": ""Within 24 hours""}","{'Priority': 'Medium', 'ETA': 'Within 24 hours'}",Medium,Within 24 hours


In [ ]:
# Dropping model_response and model_response_parsed columns
final_data_3 = data_with_parsed_model_output_3.drop(['model_response','model_response_parsed'], axis=1)
final_data_3.head()

,support_tick_id,support_ticket_text,Priority,ETA
0,ST2023-006,My internet connection has significantly slowe...,High,ASAP
1,ST2023-007,Urgent help required! My laptop refuses to sta...,High,ASAP
2,ST2023-008,I've accidentally deleted essential work docum...,High,ASAP
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Medium,Within 24 hours
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Medium,Within 24 hours


In [ ]:
final_data_3 = pd.concat([final_data_3,final_data_2[["Category","Tags"]]],axis=1)

In [ ]:
final_data_3 = final_data_3[["support_tick_id","support_ticket_text","Category","Tags","Priority","ETA"]]

In [ ]:
final_data_3

,support_tick_id,support_ticket_text,Category,Tags,Priority,ETA
0,ST2023-006,My internet connection has significantly slowe...,Technical issues,"[Internet, Connection, Slowdown, Disconnection]",High,ASAP
1,ST2023-007,Urgent help required! My laptop refuses to sta...,Hardware issues,"[Urgent, Hardware failure, Laptop start-up, Pr...",High,ASAP
2,ST2023-008,I've accidentally deleted essential work docum...,Data recovery,"data_loss, document, recovery",High,ASAP
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Technical issues,"[Wi-Fi, Signal strength, Troubleshooting]",Medium,Within 24 hours
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Technical issues,"[Battery, Smartphone, Draining, Technical Issues]",Medium,Within 24 hours
5,ST2023-011,I'm locked out of my online banking account an...,Technical issues,"[Account Access, Password Reset, Online Bankin...",High,ASAP
6,ST2023-012,"My computer's performance is sluggish, severel...",Technical issues,"[performance, optimization, computer, work]",High,Within 24 hours
7,ST2023-013,I'm experiencing a recurring blue screen error...,Hardware issues,"[Blue Screen, Crash, Hardware Issue]",High,ASAP
8,ST2023-014,My external hard drive isn't being recognized ...,Data recovery,"[Data Recovery, External Hard Drive]",High,ASAP
9,ST2023-015,The graphics card in my gaming laptop seems to...,Hardware issues,"[Graphics Card, Hardware Issue, Gaming Perform...",High,Within 24 hours


## Task 4 - Creating a Draft Response

In [ ]:
# creating a copy of the data
data_4 = data.copy()

In [ ]:
def response_4(prompt,ticket,category,tags,priority,eta):
    start = time.time()
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      Category : {category}
      Tags : {tags}
      Priority: {priority}
      ETA: {eta}
      A:
      """,
      max_tokens=80,
      stop=["Q:", "\n"],
      temperature=0.2,
      echo=False,
    )
    end = time.time()

    temp_output = model_output["choices"][0]["text"]


    return temp_output

In [ ]:
prompt_4 = """
    You are provided a Support ticket, Category of the ticket,Tags,the Priority with which it has to be resolved, and the ETA for resolution.
    Based on this information, draft a response that can be shared with the customer.

    Instructions:
    1. Carefully read the text of the support ticket and consider the overall sentiment of the review
    2. Use the tags and category for summarizing.
    3. The ETA should be mentioned in the response to the customer
    4. The response should be empathetic in nature
    5. The number of words in the response should be less than 48.

    Return only the response that has to be shared with the customer, no additional information should be present in the output.
"""

In [ ]:
#Applying generate_llama_response function on support_ticket_text column
start = time.time()
data_4['model_response'] = final_data_3[['support_ticket_text','Category','Tags','Priority','ETA']].apply(lambda x: response_4(prompt_4, x[0],x[1],x[2],x[3],x[4]),axis=1)
end = time.time()

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


In [ ]:
print("Time taken",(end-start))

Time taken 955.6914415359497


In [ ]:
data_4['model_response'].values

array([" We're truly sorry for the inconvenience you've been experiencing with your internet connection. Our team is working diligently to identify and resolve this issue as a priority (High). We understand the importance of a stable connection, especially while working from home. An estimated resolution time (ETA) is ASAP. Thank you for bringing this to our attention.",
       " We're truly sorry for the inconvenience you're experiencing with your laptop not starting. Our team will prioritize this issue as high priority and work diligently to resolve it ASAP, so you can meet your presentation deadline. Thank you for bringing this to our attention.",
       ' We deeply apologize for your data loss incident. Our team will prioritize your data recovery request (High Priority) and work towards an ASAP resolution. Kindly avoid using your device until the process is complete to prevent overwriting any potential recoverable data. Thank you for bringing this to our attention.',
       ' Dear 

In [ ]:
i = 2
print(data_4.loc[i, 'support_ticket_text'])

I've accidentally deleted essential work documents, causing substantial data loss. I understand the need to avoid further actions on my device. Can you please prioritize the data recovery process and guide me through it?


In [ ]:
print(data_4.loc[i, 'model_response'])

 We deeply apologize for your data loss incident. Our team will prioritize your data recovery request (High Priority) and work towards an ASAP resolution. Kindly avoid using your device until the process is complete to prevent overwriting any potential recoverable data. Thank you for bringing this to our attention.


In [ ]:
final_data_4 = pd.concat([final_data_3,data_4["model_response"]],axis=1)

In [ ]:
final_data_4.rename(columns={"model_response":"Response"},inplace=True)

In [ ]:
final_data_4

,support_tick_id,support_ticket_text,Category,Tags,Priority,ETA,Response
0,ST2023-006,My internet connection has significantly slowe...,Technical issues,"[Internet, Connection, Slowdown, Disconnection]",High,ASAP,We're truly sorry for the inconvenience you'v...
1,ST2023-007,Urgent help required! My laptop refuses to sta...,Hardware issues,"[Urgent, Hardware failure, Laptop start-up, Pr...",High,ASAP,We're truly sorry for the inconvenience you'r...
2,ST2023-008,I've accidentally deleted essential work docum...,Data recovery,"data_loss, document, recovery",High,ASAP,We deeply apologize for your data loss incide...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Technical issues,"[Wi-Fi, Signal strength, Troubleshooting]",Medium,Within 24 hours,"Dear Valued Customer,"
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Technical issues,"[Battery, Smartphone, Draining, Technical Issues]",Medium,Within 24 hours,"Dear Valued Customer,"
5,ST2023-011,I'm locked out of my online banking account an...,Technical issues,"[Account Access, Password Reset, Online Bankin...",High,ASAP,We're truly sorry for the inconvenience you'v...
6,ST2023-012,"My computer's performance is sluggish, severel...",Technical issues,"[performance, optimization, computer, work]",High,Within 24 hours,"Dear Valued Customer,"
7,ST2023-013,I'm experiencing a recurring blue screen error...,Hardware issues,"[Blue Screen, Crash, Hardware Issue]",High,ASAP,"Dear Valued Customer,"
8,ST2023-014,My external hard drive isn't being recognized ...,Data recovery,"[Data Recovery, External Hard Drive]",High,ASAP,We deeply apologize for the inconvenience cau...
9,ST2023-015,The graphics card in my gaming laptop seems to...,Hardware issues,"[Graphics Card, Hardware Issue, Gaming Perform...",High,Within 24 hours,"Dear Valued Customer,"


## Model Output Analysis

In [ ]:
# Creating a copy of the dataframe of task-4
final_data = final_data_4.copy()

In [ ]:
final_data['Category'].value_counts()    # complete the code with the column name for the column containing ticket categories

Technical issues    9
Data recovery       7
Hardware issues     5
Name: Category, dtype: int64

* The most common support category is "Technical Issues" with 9 occurrences, followed by "Data Recovery" with 7 occurrences, and "Hardware Issues" is the third most common category with 5 occurrences.

In [ ]:
final_data["Priority"].value_counts()

High      19
Medium     2
Name: Priority, dtype: int64

* The model identified 19 tickets to be of high priority and 2 to be of medium priority.
* No tickets were assigned as low priority.

In [ ]:
final_data["ETA"].value_counts()

ASAP               14
Within 24 hours     5
Within 48 hours     1
Within 4 hours      1
Name: ETA, dtype: int64

* The presence of an "ASAP" ETA for tickets suggests the existence of critical issues that require immediate attention.

Let's dive in a bit deeper here.

In [ ]:
final_data.groupby(['Category', 'ETA']).support_tick_id.count()

Category          ETA            
Data recovery     ASAP               7
Hardware issues   ASAP               3
                  Within 24 hours    1
                  Within 48 hours    1
Technical issues  ASAP               4
                  Within 24 hours    4
                  Within 4 hours     1
Name: support_tick_id, dtype: int64

- Data Recovery:

   - All the tickets in the "Data Recovery" category have an ETA of "ASAP".

- Hardware Issues:

   - The majority of tickets in the "Hardware Issues" category have an ETA of "ASAP"
   - There are also tickets with ETAs of "Within 24 hours" and "Within 48 hours"

- Technical Issues:

   - Tickets in the "Technical Issues" category have varied ETAs.
   - The most common ETA is "ASAP".

- ETA Distribution:

   - "ASAP" is a common ETA across different categories.
   - There's a mix of shorter and longer ETAs, indicating varying levels of urgency for issue resolution.

- Priority

    - There are no support tickets with low priority, which is unlikely in the real world.

## Actionable Insights and Recommendations

<font size=6 color='blue'>Power Ahead</font>
___